Towards Fault Tolerant Compiling
-----------
Recent experiments demonstrating that error corrected logical qubits can already achieve higher fidelities than physical qubits have us excited about Fault Tolerant (FT) quantum compilation [1, 2]. In this tutorial, we will highlight some crucial differences between synthesis of NISQ and FT circuits, and discuss how BQSKit can be used to target this new domain.

Contents
-----------
* [Introduction](#introduction)
* [Single Qubit Gates](#single-qubit-gates)
* [FT Synthesis in BQSKit](#fault-tolerant-synthesis-in-bqskit)
    * [Search-based Synthesis](#search-based-synthesis)
    * [Synthesis by Diagonalization](#synthesis-by-diagonalization)

Introduction
--------------
When targeting near term devices, compilers are most concerned about transforming circuits so that successful execution is maxized while conforming to the underlying machine's physical constraints. Typically this means that circuits are optimized to use as few multi-qudit gates as possible, requires that circuits be transpiled to a particular gate set, and demands mapping to a restrictive topology. BQSKit provides techniques for addressing each of these aspects of the compilation pipeline.

Fault tolerant computation poses different challenges. Unlike in the NISQ case, FT compilation assumes that robust error corrected qubits store information. This feat is accomplished by adding a new layer to the quantum computing stack: a quantum error correcting code [3, 4, 5]. This new protection does not come for free. For codes of interest, implementing arbitrary unitary operations on logical qubits becomes much more difficult. This is because these code admit restrictive, discrete, gate sets, whereas physical qubits can be made to implement continuous rotations. To make matters worse, only a subset of the logical gates needed for universal computation can be easily implemented. These gates are in the Clifford group (generated by $\{H, S, CNOT\}$), which contains some familiar faces such as the Pauli matrices ($\{X, Y, Z\}$).

To make our logical gate set universal, we need to add a single non-Clifford gate to the mix [6]. The most popular non-Clifford gate considered is the $T$ gate, which we define as 
$$
T = \begin{bmatrix}
1 & 0 \\
0 & e^{i\frac{\pi}{4}}
\end{bmatrix}
$$
Even though we can't implement the $T$ gate directly like we can Clifford gates, teleportation lets us inject special resource states into our circuit that can be used to implement non-Clifford gates. We can prepare these special "magic" states using a process called magic state distillation [7]. Because there is so much that must happen behind the scenes to make sure that non-Clifford gates can be executed, they typically are considered on the order of $100$-$1000\times$ more expensive than their Clifford cousins. This marks a huge departure from the optimization objective in NISQ-centric synthesis algorithms. Whereas before the primary concern was minimizing multi-qubit gates (such as $CNOT$), now the goal is to minimize non-Clifford gates (such as $T$).

Now we can consider the Clifford+$T$ gate set, which is simply $\{H, T, CNOT\}$ (note that $T^2=S$ and $T^4=Z$). Equipped with this gate set, the Solovay-Kitaev theorem tells us that we can implement an approximation of any single-qubit unitary up to a precision of $\epsilon$ in $O(log^{3.97}(\frac{1}{\epsilon}))$ gates. Along with the $CNOT$ gate, this gives us the ability to approximate any $n$-qubit operation in a fault tolerant way!

Single Qubit Gates
--------


In practice, the algorithm given by the Solovay-Kitaev theorem results in long sequences of gates when approximating single-qubit unitaries. The state of the art for ancilla free approximate synthesis comes from the Ross-Selinger "gridsynth" algorithm [8]. This algorithm produces optimal length sequences of Clifford+$T$ gates approximating $R_Z(\theta)$ rotations. Because any single-qubit unitary can be written as
$$
    U = R_Z(\theta) \sqrt{X} R_Z(\phi) \sqrt{X} R_Z(\lambda)
$$
which means that gridsynth lets us implement arbitrary single-qubit rotations that are at most only $3\times$ away from optimal. Unless $R_Z(\theta)$ corresponds to gates already in our Clifford+$T$ gate set ($Z$, $S$, or $T$), each $R_Z(\theta)$ is decomposed into a sequence of around $25\log\frac{1}{\epsilon}$ gates (of which about $\frac{1}{3}$ are $T$ gates). We will make use of the fact that we can optimally compile $R_Z(\theta)$ rotations in our approach to implementing multi-qubit unitaries in FT gate sets.

Fault Tolerant Synthesis in BQSKit
--------
Synthesis algorithms targeting the Clifford+$T$ gate set suffer a new challenge: how do you implement continuous unitaries in a discrete gate set? We will present two methods of addressing this issue: 1) search-based techniques and 2) constrained optimization.

### Search-Based Synthesis
Search-based synthesis algorithms search through the discrete space of candidate circuits. Although the search space grows as $O(|\text{gateset}|^{\text{depth}})$, this approach can still be rather effective in certain cases.

Say we want to synthesize the following unitary:

In [1]:
from numpy import array

easy_target = array([
    [0.70710678, 0, 0.70710678, 0],
    [0, 0.5+0.5j, 0, 0.5+0.5j],
    [0.5+0.5j, 0, -0.5-0.5j, 0],
    [0, 0.70710678, 0, -0.70710678],
])

We can perform A* search over discrete gate sets in BQSKit by using a `DiscreteLayerGenerator`. Furthermore, we want to make sure synthesis gives us a very good implementation. We're going to set the `success_threshold=1e-8` to ensure this. The following block should take around 15 seconds to run.

In [3]:
from utils import DiscreteLayerGenerator

from bqskit.passes.synthesis import QSearchSynthesisPass
from bqskit.compiler import Compiler
from bqskit import Circuit

success_threshold = 1e-8
layer_generator = DiscreteLayerGenerator()
synthesis_pass = QSearchSynthesisPass(
    success_threshold=success_threshold,
    layer_generator=layer_generator,
)

circuit = Circuit.from_unitary(easy_target)

with Compiler() as compiler:
    circuit = compiler.compile(circuit, [synthesis_pass])

for op in circuit:
    print(op)


HGate@(0,)
CNOTGate@(0, 1)
TGate@(1,)
CNOTGate@(0, 1)


In this case, we were able to find a solution! Now let's say we want to try a slightly different unitary:

In [4]:
hard_target = array([
   [0.70370187-0.06930858j, 0, 0.70370187-0.06930858j, 0],
   [0, 0.70370187+0.06930858j, 0, 0.70370187+0.06930858j],
   [0.70370187+0.06930858j, 0, -0.70370187-0.06930858j, 0],
   [0, 0.70370187-0.06930858j, 0, -0.70370187+0.06930858j],
])

What happens if you try to use the `DiscreteLayerGenerator` to synthesize this unitary?

(Note: you may have to restart the kernel after this part)

In [ ]:
# Use the DiscreteLayerGenerator to try synthesizing hard_target

### Synthesis by Diagonalization
Pure search-based synthesis has its limits. If the unitary is not easily implementable in the discrete gate set provided to the `DiscreteLayerGenerator`, the synthesis algorithm may spend lots of time checking different candidates before eventually timing out. There are lots of techniques that can push the boundaries of what's possible with pure discrete search, but these techniques will always be limited by the sheer size of the solution space. Finding solutions to a precision of `success_threshold=1e-3` might be possible in certain cases, but beyond that, for high precision, a new technique is needed.

To approach this problem, we're going to make an assumption: finding a solution circuit requires $R_Z(\theta)$ gates. Making this assumption means that our synthesis algorithm is unlikely to find optimal solutions, but a good although suboptimal solution with $R_Z(\theta)$ gates is better than no solution at all!

How can we modify the search-based synthesis framework to handle unitaries whose implementation require $R_Z(\theta)$ gates? Using `QSearch` then converting every `U3Gate` to a sequence of `RZGate`s and `SqrtXGate`s is one option, which will be the topic of the next section, but there's another way.

We're going to assume that the circuit which implements our `hard_target` unitary is in the form:
$$
    C(\bar\theta) = L \times D_{\bar\theta} \times R
$$
where $D_{\bar\theta}$ is a diagonal unitary, and the $L$ and $R$ subcircuits can be found by our search-based synthesis algorithm. We relax the need for our search-based synthesis algorithm to find sequences of gates that implement our target, to just finding gates that diagonalize our target.

In [30]:
success_threshold = 1e-8
# double_headed=True means we will try to diagonalize the target
layer_generator = DiscreteLayerGenerator(double_headed=True)
synthesis_pass = QSearchSynthesisPass(
    success_threshold=success_threshold,
    layer_generator=layer_generator,
)

circuit = Circuit.from_unitary(hard_target)

# Diagonalize
with Compiler() as compiler:
    circuit = compiler.compile(circuit, synthesis_pass)

from utils.diagonal_synth import replace_pauliz
# Decompose diagonal component
circuit = replace_pauliz(circuit)

for op in circuit:
    if op.num_params > 0:
        print(f'{op} - {op.params}')
    else:
        print(op)

HGate@(0,)
CNOTGate@(0, 1)
RZGate@(1,) - [0.19634952754588517]
CNOTGate@(0, 1)


__Question:__ What similarities do you see between the outputs of synthesis for `easy_target` and `hard_target`?

__Question:__ Assume that the best implementation of `hard_target` corresponds to the results from above, but with the $R_Z$ operation converted to ${H,T}$ using gridsynth. Approximately how many candidate circuits might need to be searched to find this circuit without diagonalization? Assume `success_threshold=1e-8`.

It turns out that there are many unitaries that appear in real quantum algorithms which are amenable to synthesis by Clifford+T diagonalization. Check out this paper if you'd like to learn about how we trained Reinforcement Learning agents to do exactly this!

## References
-------------
[1] https://arxiv.org/abs/2312.03982, D. Bluvstein, et al., "Logical quantum processor based on reconfigurable atom arrays"
<br>[2] https://arxiv.org/abs/2404.02280, M. P. da Silva, et al., "Demonstration of logical qubits and repeated error correction with better-than-physical error rates"
<br>[3] https://journals.aps.org/pra/abstract/10.1103/PhysRevA.52.R2493, P. Shor, "Scheme for reducing decoherence in quantum computer memory"
<br>[4] https://arxiv.org/abs/quant-ph/9601029, A. Steane, "Multiple Particle Interference and Quantum Error Correction"
<br>[5] https://arxiv.org/abs/quant-ph/9811052, S. Bravyi, A. Kitaev, "Quantum codes on a lattice with boundary"
<br>[6] https://iopscience.iop.org/article/10.1070/RM1997v052n06ABEH002155/meta, A. Kitaev, "Quantum computations: algorithms and error correction"
<br>[7] https://arxiv.org/abs/quant-ph/0403025, S. Bravyi, A. Kitaev, "Universal Quantum Computation with ideal Clifford gates and noisy ancillas"
<br>[8] https://arxiv.org/abs/1403.2975, N. Ross, P. Selinger, "Optimal ancilla-free Clifford+T approximations of z-rotations"
<br>[9] DVNU PAPER